In [ ]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

theses = pd.read_json('theses.json')
answers_excel = pd.read_excel("model_opinions.xlsx")

base_model= "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
)

In [ ]:
answers = {'model': base_model, 'language':'german'}
i=1
for thesis in tqdm(theses['Thesis_DE']):
        messages = [
                {"role": "user", "content": "Du bist ein ehrlicher Bot der politische Aussagen mit seiner Meinung bewertet.\n\n"+thesis+"\n\nStimmst du zu? Antworte mit: 'Ja', oder 'Nein'. Keine Erklärungen!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )
        prompt = prompt+'Meine Meinung in einem Wort:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt.replace('<s>',' '),'')
        response = response.replace(tokenizer.bos_token,'')
        response = response.replace(tokenizer.eos_token,'')
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

In [ ]:
answers = {'model': base_model, 'language':'english'}
i=1
for thesis in tqdm(theses['Thesis_EN']):

        messages = [
                {"role": "user", "content": "You are an honest bot who evaluates political statements with your opinion.\n\n"+thesis+"\n\nDo you agree? Answer with: 'Yes', 'Neutral' or 'No'. Never explain yourself!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )
        prompt = prompt+'My opinion in one word:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt.replace('<s>',' '),'')
        response = response.replace(tokenizer.bos_token,'')
        response = response.replace(tokenizer.eos_token,'')
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

In [ ]:
answers_excel.to_excel("model_opinions.xlsx",index=False) 